In [ ]:
import os
import pandas as pd
import urllib
import zipfile
import xmltodict

In [ ]:
REMOTE_PATH = "http://adrien.barbaresi.eu/corpora/speeches/"
REMOTE_FILE = "German-political-speeches-2018-release.zip"
REMOTE_URL = REMOTE_PATH + REMOTE_FILE

DATA_FOLDER = "data"
DATASET = "Bundesregierung.xml"

In [1]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' + directory)

def download_and_extract_data():
    createFolder(DATA_FOLDER)
    zip_path = os.path.join(DATA_FOLDER, REMOTE_FILE)
    urllib.request.urlretrieve(REMOTE_URL, zip_path)
    with zipfile.ZipFile(zip_path) as file:
        file.extract(DATASET, path = DATA_FOLDER)

def load_cached_speeches_or_download():
    zip_path = os.path.join(DATA_FOLDER, REMOTE_FILE)
    if not os.path.isfile(zip_path):
        download_and_extract_data()
    file = os.path.join(DATA_FOLDER, DATASET) 
    with open(file, mode="rb") as file:
        xml_document = xmltodict.parse(file)
        text_nodes = xml_document['collection']['text']
        persons = [t['@person'] for t in text_nodes]
        speeches = [t['rohtext'] for t in text_nodes]
        return pd.DataFrame({'person' : persons, 'speech' : speeches})